In [ ]:
from PropertyInvestment import PropertyInvestment
from StockInvestment import StockInvestment
from plotting import PlotSeries, plot


TIME_FRAME_IN_YEARS = 50
INITIAL_CASH = 50000
LOAN_INTEREST_RATE = 0.036
INITIAL_LOAN_AMORTIZATION_RATE = 0.01
AVERAGE_STOCK_RETURN = 0.1
MONTHLY_AVAILABLE_CASH = 2500
# Kaufpreis und Mietpreis sind für Eschborn und stammen aus dem Internet
RENT_PER_SQUARE_METER = 14.49
PRICE_PER_SQUARE_METER = 4130

area = 157
property_rent = area * RENT_PER_SQUARE_METER
property_price = area * PRICE_PER_SQUARE_METER

valueSeriesList = []
stockValueSeriesList = []
propValueSeriesList = []
for house_purchase_delay in range(0, 60, 10):
    valueSeries = PlotSeries(
        "House purchase after " + str(house_purchase_delay) + " Years"
        if house_purchase_delay > 0
        else "House purchase immediately"
    )
    stockValueSeries = PlotSeries(
        "House purchase after " + str(house_purchase_delay) + " Years"
        if house_purchase_delay > 0
        else "House purchase immediately"
    )
    propValueSeries = PlotSeries(
        "House purchase after " + str(house_purchase_delay) + " Years"
        if house_purchase_delay > 0
        else "House purchase immediately"
    )
    stock_inv = StockInvestment(AVERAGE_STOCK_RETURN, INITIAL_CASH)
    prop_inv = None
    for i in range(TIME_FRAME_IN_YEARS * 12 + 2):
        cash = MONTHLY_AVAILABLE_CASH
        if i < house_purchase_delay * 12:
            stock_inv.deposit(cash - property_rent)
        elif i == house_purchase_delay * 12:
            required_cash = property_price - cash
            redeemed_cash = stock_inv.attempt_withdrawal(required_cash)
            cash_for_property = redeemed_cash + cash
            prop_inv = PropertyInvestment(
                property_price,
                LOAN_INTEREST_RATE,
                INITIAL_LOAN_AMORTIZATION_RATE,
                cash_for_property,
            )
        else:
            stock_inv.deposit(cash - prop_inv.get_rate())

        valueSeries.put_value(
            (
                stock_inv.get_value()
                + (prop_inv.get_total_amortization() if prop_inv is not None else 0)
            )
            / 1000000
        )
        stockValueSeries.put_value(stock_inv.get_value() / 1000000)
        propValueSeries.put_value(
            prop_inv.get_total_amortization() / 1000000 if prop_inv is not None else 0
        )
        stock_inv.step()
        if prop_inv is not None:
            prop_inv.step()
    valueSeriesList.append(valueSeries)
    stockValueSeriesList.append(stockValueSeries)
    propValueSeriesList.append(propValueSeries)

plot(
    {
        "title": "Total Value over Time",
        "xlabel": "Year",
        "ylabel": "Value in Mio €",
        "series": valueSeriesList,
    }
)
plot(
    {
        "title": "Stock Value over Time",
        "xlabel": "Year",
        "ylabel": "Value in Mio €",
        "series": stockValueSeriesList,
    }
)
plot(
    {
        "title": "Property Value over Time",
        "xlabel": "Year",
        "ylabel": "Value in Mio €",
        "series": propValueSeriesList,
    }
)